In [1]:
import threading
import queue
import time
import keyboard
import queue
from model import *
import pytchat
from data import *

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

# Define a function to set global variables
def set_globals(args):
    global PATIENCE, BATCH_SIZE,DATASET, BLOCK_SIZE, DROPOUT, LEARNING_RATE, EPOCHS, FRAMES_GENERATE, TRAIN, EVAL_EVERY, CHECKPOINT_PATH, L1_LAMBDA, L2_REG, FINETUNE, FINE_TUNING_LR, FINE_TUNING_EPOCHS, PENALTY, LATENT_VIS_EVERY, notes,USE_MDN
    BATCH_SIZE = args.BATCH_SIZE
    BLOCK_SIZE = args.BLOCK_SIZE
    DROPOUT = args.DROPOUT
    LEARNING_RATE = args.LEARNING_RATE
    EPOCHS = args.EPOCHS
    FRAMES_GENERATE = args.FRAMES_GENERATE
    TRAIN = args.TRAIN
    EVAL_EVERY = args.EVAL_EVERY
    CHECKPOINT_PATH = args.CHECKPOINT_PATH
    L1_LAMBDA = args.L1_LAMBDA
    L2_REG = args.L2_REG
    FINETUNE = args.FINETUNE
    FINE_TUNING_LR = args.FINE_TUNING_LR
    FINE_TUNING_EPOCHS = args.FINE_TUNING_EPOCHS
    PENALTY = args.PENALTY
    LATENT_VIS_EVERY = args.LATENT_VIS_EVERY
    USE_MDN = args.USE_MDN
    DATASET = args.DATASET
    notes = args.notes
    PATIENCE = args.PATIENCE
    
    # ---------------------------------
    notes = f"""Proto8 - trying to adapt Pette et al 2019, addign latent visualisation and analysing latent space. Might be slow, maybe take this out when live.

    
    Added MDN layer to model.
    
    All data, added 10% noise to emotions so model is less stuck. With LeakyRelu
    Loss = mse_loss(keypoints) + mse_loss(emotions) because before output emotions ( which feature was added to keypoint features) were not being matched to input emotions
    No penalty.

    Added dropout to keypoints, also changed input to emotion linear to x and not just emotion (emotion + keypoints)
    Taking extra dropout for emotions and keypoints out, because want model to rely on both equally so what's the point

    dropout keypoints and dropout emotion is currently equal but might change this.

    Emotions and keypoints are multimodal and added separately, but features are added in block processing using +.


    Got rid of both L1 and L2, increasing dropout because model acting weird, this is now delta + coord. 
    Delta is between next frame and current frame. So current frame is previous coord+previous delta. Last frame's delta is 0. 
    
    {BATCH_SIZE} batch size, {BLOCK_SIZE} block size, {DROPOUT} dropout, {LEARNING_RATE} learning rate, {EPOCHS} epochs, {FRAMES_GENERATE} frames generated, {TRAIN} train, {EVAL_EVERY} eval every, {CHECKPOINT_PATH} checkpoint path, {L1_LAMBDA} L1 lambda, {L2_REG} L2 reg"""
    # ---------------------------------
    
    # Print the values using f-string for formatting
    print(f"""
    Batch size set to: {BATCH_SIZE}
    Block size set to: {BLOCK_SIZE}
    Dropout rate set to: {DROPOUT}
    Learning rate set to: {LEARNING_RATE}
    Number of epochs set to: {EPOCHS}
    Frames to generate set to: {FRAMES_GENERATE}
    Training mode set to: {TRAIN}
    Evaluation every set to: {EVAL_EVERY}
    Checkpoint path set to: {CHECKPOINT_PATH}
    L1 regularization lambda set to: {L1_LAMBDA}
    L2 regularization lambda set to: {L2_REG}
    Fine-tuning mode set to: {FINETUNE}
    Fine-tuning learning rate set to: {FINE_TUNING_LR}
    Fine-tuning epochs set to: {FINE_TUNING_EPOCHS}
    Penalty flag set to: {PENALTY}
    Latent visualization every set to: {LATENT_VIS_EVERY}
    Use MDN flag set to: {USE_MDN}
    Dataset set to: {DATASET}
    Patience: {PATIENCE}
    """)
    

# initialise model------------------------------------------------------------


args = argparse.Namespace(
        BATCH_SIZE=1,
        BLOCK_SIZE=16,
        DROPOUT=0.2,
        LEARNING_RATE=0.0001,
        EPOCHS=100000,
        FRAMES_GENERATE=150,
        TRAIN=False,
        EVAL_EVERY=1000,
        CHECKPOINT_PATH="checkpoints/proto8_checkpoint_temp.pth",
        L1_LAMBDA=None,
        L2_REG=0.0,
        FINETUNE=False,
        FINE_TUNING_LR=1e-5,
        FINE_TUNING_EPOCHS=100000,
        PENALTY=False,
        LATENT_VIS_EVERY=1000,
        USE_MDN = True,
        DATASET = "all",
        PATIENCE = 10,
        
        # NOTES---------------------------------
        notes = f"""Proto8 - trying to adapt Pette et al 2019, addign latent visualisation and analysing latent space. Might be slow, maybe take this out when live.
        
        Added MDN to increase variance of output as Bishop et al 1994. and Alemi et al 2017.
        
        MEED data only

        All data, added 10% noise to emotions so model is less stuck. With LeakyRelu
        Loss = mse_loss(keypoints) + mse_loss(emotions) because before output emotions ( which feature was added to keypoint features) were not being matched to input emotions
        No penalty.

        Added dropout to keypoints, also changed input to emotion linear to x and not just emotion (emotion + keypoints)
        Taking extra dropout for emotions and keypoints out, because want model to rely on both equally so what's the point

        dropout keypoints and dropout emotion is currently equal but might change this.

        Emotions and keypoints are multimodal and added separately, but features are added in block processing using +.


        Got rid of both L1 and L2, increasing dropout because model acting weird, this is now delta + coord. 
        Delta is between next frame and current frame. So current frame is previous coord+previous delta. Last frame's delta is 0. 
        """
    )

# If args are provided, use those; otherwise, parse from command line
if args is None:
    args = parse_args()

# Set the global variables based on args
set_globals(args)

# Set global variables

processed_data= prep_data(dataset=args.DATASET)
# global train_data,train_emotions, val_data, val_emotions, frame_dim, max_x, min_x, max_y, min_y, max_dx, min_dx, max_dy, min_dy, threshold
train_data, train_emotions, val_data, val_emotions, frame_dim, max_x, min_x, max_y, min_y, max_dx, min_dx, max_dy, min_dy, threshold = processed_data

# create model
# global m
m = MotionModel(input_dim=frame_dim, output_dim=frame_dim,emotion_dim=7, blocksize=args.BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=args.DROPOUT)
m = m.to(device)

optimizer = torch.optim.Adam(m.parameters(), lr=args.LEARNING_RATE, weight_decay=args.L2_REG)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
# Load the model
print('Loading model...')


m, optimizer, scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, args.CHECKPOINT_PATH,scheduler)
print(f"Model {train_seed} loaded from {args.CHECKPOINT_PATH} (epoch {epoch}, loss {loss:.6f})")


# Functions
def normalise_generated(unnorm_out, max_x, min_x, max_y, min_y, max_dx, min_dx, max_dy, min_dy): 
    norm_out = []
    
    for frame in unnorm_out:
        norm_frame = []
        
        # Normalize the first 50 values (absolute x and y coordinates)
        for i in range(0, 50, 2):
            unnormalized_x = frame[i]
            unnormalized_y = frame[i+1]
            
            norm_x = 2 * (unnormalized_x - min_x) / (max_x - min_x) - 1
            norm_y = 2 * (unnormalized_y - min_y) / (max_y - min_y) - 1
            
            norm_frame.extend([norm_x, norm_y])
        
        # Normalize the second 50 values (x and y deltas)
        for i in range(50, 100, 2):
            unnormalized_dx = frame[i]
            unnormalized_dy = frame[i+1]
            
            norm_dx = 2 * (unnormalized_dx - min_dx) / (max_dx - min_dx) - 1
            norm_dy = 2 * (unnormalized_dy - min_dy) / (max_dy - min_dy) - 1
            
            norm_frame.extend([norm_dx, norm_dy])
        
        # Append the emotion encoding without normalizing
    
        norm_out.append(norm_frame)
        
    return norm_out


# Initial setup
shared_data = {
    'average_scores': [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

# different from normal emotion labels - matches the sentiment analyser
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

emotion_data = {emotion: {"score": 0.0, "count": 0} for emotion in emotion_labels}

chat = pytchat.create(video_id="gCNeDWCI0vo")
FRAMES_GENERATE = 150
terminate_threads = False

# This queue will hold the batches ready for visualization
viz_queue = queue.Queue()


def process_chat_message(c):
    """Process a chat message and update emotion scores."""
    print(f"{c.datetime} [{c.author.name}]- {c.message}")
    result = pipe(c.message)  # Assuming pipe() returns emotion prediction
    print(result)

    detected_emotion = result[0]['label']

    # Reset the counter for the detected emotion and boost its score
    emotion_data[detected_emotion]["count"] = 0
    score = result[0]['score']
    emotion_data[detected_emotion]["score"] = min(1, emotion_data[detected_emotion]["score"] + score)

    # Decay scores for other emotions and increase their counters
    for emotion, data in emotion_data.items():
        if emotion != detected_emotion:
            data["count"] += 1
            if data["count"] >= 5:
                data["score"] = 0
            else:
                data["score"] *= 0.5  # or any other decay factor you prefer

    # Update average scores
    for i, emotion in enumerate(emotion_labels):
        shared_data['average_scores'][i] = emotion_data[emotion]["score"]

    print("Average scores:", shared_data['average_scores'])

# Batch generation function
def generate_new_batch(last_frame=None):
    """Generate a new batch based on the current average scores."""
    # If initial_data is None or empty, initialize with default values
    if last_frame is None:
        print('LAST FRAME IS NONE')
        last_frame = torch.randn(1,5, 100).to(device)  # initialise with noise

    last_frames = last_frame[0][-3:]
    norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)
    new_input = torch.tensor([norm_last_frames]).to(device).float()
    emotion_in = torch.tensor([shared_data['average_scores']]).to(device).float()

    # Generate the new frames
    generated_keypoints, generated_emotion = m.generate(new_input, emotion_in, FRAMES_GENERATE)
    
    emotion_vectors = (emotion_in, generated_emotion)
    return unnormalise_list_2D(generated_keypoints, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y), emotion_vectors

def generate_batches_periodically(period=2, last_frame=None):
    # initialise with last_frame = None
    while not terminate_threads:  
        time.sleep(period)
        unnorm_out, emotion_vectors = generate_new_batch(last_frame)
        viz_queue.put((unnorm_out, emotion_vectors))  
        last_frame = unnorm_out
        

def visualise(unnorm_out, emotion_vectors):
    # visualize
    emotion_in, generated_emotion = emotion_vectors 
    emotion_vectors = (emotion_in[0], generated_emotion[0]) #quick fix
    
    visualise_skeleton(unnorm_out[0], max_x, max_y, emotion_vectors,max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'{EPOCHS}_main_test',train_seed=train_seed,delta=False,destroy=False)

def visualise_batches():
    while not terminate_threads:  # Check the global termination flag
        batch = viz_queue.get()  # Get the tuple from the queue
        if batch is None:  # Check if the thread should terminate
            break
        unnorm_out, emotion_vectors = batch  # Unpack the tuple
        visualise(unnorm_out, emotion_vectors)

# Start the threads
visualisation_thread = threading.Thread(target=visualise_batches, daemon=True)
generation_thread = threading.Thread(target=generate_batches_periodically, args=(10,), daemon=True)

visualisation_thread.start()
generation_thread.start()


# Process chat messages
while chat.is_alive():
    if keyboard.is_pressed('esc'):  # Check if ESC key is pressed
        terminate_threads = True
        viz_queue.put(None)  # Put a None in the queue to signal the visualisation thread to terminate
        break  # Exit the main loop
    for c in chat.get().sync_items():
        process_chat_message(c)

cv2.destroyAllWindows()

# Wait for threads to finish if needed
visualisation_thread.join()
generation_thread.join()

C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis

    Batch size set to: 1
    Block size set to: 16
    Dropout rate set to: 0.2
    Learning rate set to: 0.0001
    Number of epochs set to: 100000
    Frames to generate set to: 150
    Training mode set to: False
    Evaluation every set to: 1000
    Checkpoint path set to: checkpoints/proto8_checkpoint_temp.pth
    L1 regularization lambda set to: None
    L2 regularization lambda set to: 0.0
    Fine-tuning mode set to: False
    Fine-tuning learning rate set to: 1e-05
    Fine-tuning epochs set to: 100000
    Penalty flag set to: False
    Latent visualization every set to: 1000
    Use MDN flag set to: True
    Dataset set to: all
    Patience: 10
    
Preparing data for all...


100%|██████████| 4102/4102 [00:03<00:00, 1084.07it/s]


Validating interpolation...
No errors found!


4102it [00:01, 2354.05it/s]
4102it [00:01, 2427.38it/s]
100%|██████████| 109/109 [00:05<00:00, 20.35it/s]


Validating interpolation...
No errors found!


...

Validating length of dkp_frames
Validating length of kp_frames
Adding deltas to frames...


4211it [00:02, 1432.14it/s]


Validating length of data after delta
frame_dim: 100
2023-11-15 18:57:42 [Poco]- Israel Forever - Forever Israel :red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart: Israel Forever - Forever Israel :red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:red_heart:🇮🇱:hundred_points:
[{'label': 'joy', 'score': 0.6289936900138855}]
Average scores: [0.0, 0.0, 0.0, 0.6289936900138855, 0.0, 0.0, 0.0]
2023-11-15 18:57:43 [XretroBox420]- They should just send Jewish People back to their homelands in Europe. They'll be treated real nicely there!!!
[{'label': 'anger', 'score': 0.8370621800422668}]
Average scores: [0.8370621800422668, 0.0, 0.0, 0.31449684500694275, 0.0, 0.0, 0.0]


Exception in thread Thread-7 (generate_batches_periodically):
Traceback (most recent call last):
  File "c:\Users\avika\anaconda3\envs\interactive_dance_thesis\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\avika\anaconda3\envs\interactive_dance_thesis\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\avika\AppData\Local\Temp\ipykernel_45352\1679118185.py", line 266, in generate_batches_periodically
  File "C:\Users\avika\AppData\Local\Temp\ipykernel_45352\1679118185.py", line 256, in generate_new_batch
  File "c:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis\model.py", line 327, in generate
    if USE_MDN:
NameError: name 'USE_MDN' is not defined


2023-11-15 18:57:47 [Iblis]- :victory_hand:
[{'label': 'neutral', 'score': 0.9853259325027466}]
Average scores: [0.4185310900211334, 0.0, 0.0, 0.15724842250347137, 0.9853259325027466, 0.0, 0.0]
2023-11-15 18:57:51 [Linda Hlatshwako]- dead hope...they should stay in lebanon
[{'label': 'neutral', 'score': 0.9353736639022827}]
Average scores: [0.2092655450105667, 0.0, 0.0, 0.07862421125173569, 1, 0.0, 0.0]


In [4]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

class OBJ:
    def __init__(self, filename):
        self.vertices = []
        self.faces = []
        self.load_obj(filename)

    def load_obj(self, filename):
        for line in open(filename, "r"):
            if line.startswith('#'): continue
            values = line.split()
            if not values: continue

            if values[0] == 'v':
                self.vertices.append(list(map(float, values[1:4])))
            elif values[0] == 'f':
                face = []
                for v in values[1:]:
                    w = v.split('/')
                    face.append(int(w[0]))
                self.faces.append(face)

    def render(self):
        glBegin(GL_TRIANGLES)
        for face in self.faces:
            for vertex in face:
                glVertex3fv(self.vertices[vertex - 1])
        glEnd()

def main():
    pygame.init()
    display = (800,600)
    pygame.display.set_mode(display, DOUBLEBUF|OPENGL)
    gluPerspective(45, (display[0]/display[1]), 0.1, 50.0)
    glTranslatef(0.0,0.0, -5)

    obj = OBJ("data/human_mesh.obj")  # Replace 'yourmodel.obj' with your file name

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

        glRotatef(1, 3, 1, 1)
        glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
        obj.render()
        pygame.display.flip()
        pygame.time.wait(10)

if __name__ == "__main__":
    main()


GLError: GLError(
	err = 1282,
	description = b'invalid operation',
	baseOperation = glRotatef,
	cArguments = (1, 3, 1, 1)
)

: 